In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
import pandas as pd
from sklearn.inspection import permutation_importance


In [59]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
test_df = pd.read_csv('../data/raw/dmml1_test.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [60]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
test_df = pd.merge(test_df,store,how="inner",on="Store ID")

In [61]:
train_df["CompetitionDistance"].replace(np.nan,train_df["CompetitionDistance"].mean(),inplace=True)
train_df.fillna(0,inplace=True)

In [62]:
train_df["StateHoliday"].replace("0",0,inplace=True)

In [63]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week
train_df['Weekend'] = np.where(train_df['DayOfWeek'].isin([6, 7]), 1, 0)  # Samstag = 6, Sonntag = 7

train_df['Quarter'] = train_df['Date'].dt.quarter
train_df['DayOfYear'] = train_df['Date'].dt.dayofyear
train_df['DayOfMonth'] = train_df['Date'].dt.day

train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))
# train_df['Season'].replace(to_replace=[1,2,3,4], value=['Winter', 'Frühling','Sommer','Herbst'], inplace=True)

# Entfernen der ursprünglichen Date-Spalte
train_df.drop('Date', axis=1, inplace=True)

In [67]:
train_df["StateHoliday"] = train_df["StateHoliday"].replace("a", 1).replace("b", 2).replace("c", 3)
train_df["StoreType"] = train_df["StoreType"].replace("a", 1).replace("b", 2).replace("c", 3).replace("d", 4)
train_df["Assortment"] = train_df["Assortment"].replace("a", 1).replace("b", 2).replace("c", 3)

In [49]:
# train_df["StateHoliday"] = train_df["StateHoliday"].replace("a","Feiertag").replace("b","Ostern").replace("c","Weihnachten").replace(0,"Kein Feiertag")

In [10]:
# train_df["StoreType"] = train_df["StoreType"].replace("a","StoreType A").replace("b","StoreTypee B").replace("c","StoreType C").replace("d","StoreType D")

In [11]:
# train_df["Assortment"] = train_df["Assortment"].replace("a","Basic").replace("b","Extra").replace("c","Extended")

In [70]:
train_df['PromoInterval'].unique()

array([2, 1, 3, 4])

In [69]:
train_df["PromoInterval"] = train_df["PromoInterval"].replace(0, 1).replace("Jan,Apr,Jul,Oct", 2).replace("Feb,May,Aug,Nov", 3).replace("Mar,Jun,Sept,Dec", 4)


In [23]:
train_df["PromoInterval"] = train_df["PromoInterval"].replace(0,"No PromoInterval")#.replace("Jan,Apr,Jul,Oct","From Jan to Oct").replace("Feb,May,Aug,Nov","From Feb to Nov").replace("Mar,Jun,Sept,Dec","From Mar to Dec")

# def myPinterval(x):
#     if x=='From Feb to Nov':
#         return([0,1,0,0,1,0,0,1,0,0,1,0])
#     elif x=='From Jan to Oct':
#         return([1,0,0,1,0,0,1,0,0,1,0,0])
#     elif x== 'From Mar to Dec':
#         return([0,0,1,0,0,1,0,0,1,0,0,1])
#     else:
#         return(np.repeat(0,12).tolist())

# proInt = train_df.PromoInterval.apply(myPinterval).tolist()
# proInt = pd.DataFrame(proInt, columns = ['PromoInt'+ str(i) for i in range(1,13)])
# train_df = train_df.drop('PromoInterval',1).join(proInt)

In [65]:
# Erstellen Sie eine temporäre Spalte, die 1 ist, wenn der Tag ein Feiertag ist, und 0 sonst
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)

# Entfernen Sie die temporäre Spalte
train_df.drop('Temp', axis=1, inplace=True)

In [71]:
train_df

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Month,Day,WeekOfYear,Weekend,Quarter,DayOfYear,DayOfMonth,Season,TagVorFeiertag,TagnachFeiertag
0,1,4,6884,716,1,1,0,0,1,1,...,4,30,18,0,2,120,30,2,0.0,0.0
1,1,3,6764,756,1,1,0,0,1,1,...,4,29,18,0,2,119,29,2,0.0,0.0
2,1,2,6861,678,1,1,0,0,1,1,...,4,28,18,0,2,118,28,2,0.0,0.0
3,1,1,6523,647,1,1,0,0,1,1,...,4,27,18,0,2,117,27,2,0.0,0.0
4,1,7,0,0,0,0,0,0,1,1,...,4,26,17,1,2,116,26,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,5194,569,1,0,0,0,1,3,...,1,5,1,1,1,5,5,1,0.0,0.0
246899,300,5,5524,646,1,0,0,1,1,3,...,1,4,1,0,1,4,4,1,0.0,0.0
246900,300,4,5563,718,1,0,0,1,1,3,...,1,3,1,0,1,3,3,1,0.0,0.0
246901,300,3,6218,753,1,0,0,1,1,3,...,1,2,1,0,1,2,2,1,1.0,0.0


In [25]:
# Erstellen Sie den OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Liste der Spalten, die encodiert werden sollen
columns_to_encode = ['PromoInterval']
# columns_to_encode = ['StateHoliday', 'StoreType', 'Assortment', 'Season', 'PromoInterval']

for column in columns_to_encode:
    # Führen Sie das One-Hot-Encoding für die Spalte durch
    column_encoded = encoder.fit_transform(train_df[[column]])

    # Erstellen Sie einen DataFrame aus dem encodierten Array
    column_df = pd.DataFrame(column_encoded, columns=encoder.categories_[0])

    # Fügen Sie die neuen Spalten zum DataFrame hinzu und entfernen Sie die alte Spalte
    train_df = pd.concat([train_df.drop(column, axis=1), column_df], axis=1)

In [76]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

In [75]:
X

,Store ID,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,...,Month,Day,WeekOfYear,Weekend,Quarter,DayOfYear,DayOfMonth,Season,TagVorFeiertag,TagnachFeiertag
0,1,4,1,1,0,0,1,1,570.0,11.0,...,4,30,18,0,2,120,30,2,0.0,0.0
1,1,3,1,1,0,0,1,1,570.0,11.0,...,4,29,18,0,2,119,29,2,0.0,0.0
2,1,2,1,1,0,0,1,1,570.0,11.0,...,4,28,18,0,2,118,28,2,0.0,0.0
3,1,1,1,1,0,0,1,1,570.0,11.0,...,4,27,18,0,2,117,27,2,0.0,0.0
4,1,7,0,0,0,0,1,1,570.0,11.0,...,4,26,17,1,2,116,26,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,1,0,0,0,1,3,9260.0,0.0,...,1,5,1,1,1,5,5,1,0.0,0.0
246899,300,5,1,0,0,1,1,3,9260.0,0.0,...,1,4,1,0,1,4,4,1,0.0,0.0
246900,300,4,1,0,0,1,1,3,9260.0,0.0,...,1,3,1,0,1,3,3,1,0.0,0.0
246901,300,3,1,0,0,1,1,3,9260.0,0.0,...,1,2,1,0,1,2,2,1,1.0,0.0


In [77]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [73]:
models=[LinearRegression(), 
        Ridge(), 
        Lasso(), 
        DecisionTreeRegressor(random_state=42), 
        GradientBoostingRegressor(random_state=42), 
        RandomForestRegressor(n_estimators=42, n_jobs=-1, random_state=42)]

model_names=["LinearRegression","Ridge","Lasso", "DecisionTreeRegressor", "GradientBoostingRegressor", "RandomForestRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}

for index in range (len(models)):
    model=models[index]
    start_time = time.time()
    model.fit(X_train,y_train)
    end_time = time.time()
    y_pre=model.predict(X_test)

    MSE.append(round(mean_squared_error(y_true=y_test,y_pred=y_pre),5))
    r_2.append(r2_score(y_true=y_test,y_pred=y_pre))
    accuracy.append((model.score(X_test,y_test))*100)
    RMSE.append(sqrt(mean_squared_error(y_true=y_test,y_pred=y_pre)))
    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time})
print("Verwendete Features: ", X_train.columns.tolist())

LinearRegression wurde trainiert und benötigte 0.7071008682250977 Sekunden
Ridge wurde trainiert und benötigte 0.04032588005065918 Sekunden


/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.859e+11, tolerance: 3.173e+08
  model = cd_fast.enet_coordinate_descent(


Lasso wurde trainiert und benötigte 3.1481778621673584 Sekunden
DecisionTreeRegressor wurde trainiert und benötigte 1.3146507740020752 Sekunden
GradientBoostingRegressor wurde trainiert und benötigte 23.581223011016846 Sekunden
RandomForestRegressor wurde trainiert und benötigte 7.758948802947998 Sekunden
Verwendete Features:  ['Store ID', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day', 'WeekOfYear', 'Weekend', 'Quarter', 'DayOfYear', 'DayOfMonth', 'Season', 'TagVorFeiertag', 'TagnachFeiertag', 'Feb,May,Aug,Nov', 'Jan,Apr,Jul,Oct', 'Mar,Jun,Sept,Dec', 'No PromoInterval']


In [74]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time
0,LinearRegression,6.935371e+06,0.567006,56.700641,2633.509285,0.707101
1,Ridge,6.936111e+06,0.566960,56.696023,2633.649704,0.040326
2,Lasso,6.945829e+06,0.566354,56.635350,2635.494055,3.148178
3,DecisionTreeRegressor,1.247064e+06,0.922143,92.214252,1116.720004,1.314651
4,GradientBoostingRegressor,4.412961e+06,0.724487,72.448714,2100.704942,23.581223
5,RandomForestRegressor,6.778967e+05,0.957677,95.767712,823.344796,7.758949


In [30]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

models=[LinearRegression(), 
        Ridge(), 
        Lasso(), 
        DecisionTreeRegressor(random_state=42), 
        GradientBoostingRegressor(random_state=42), 
        RandomForestRegressor(n_estimators=42, n_jobs=-1, random_state=42)]

model_names=["LinearRegression","Ridge","Lasso", "DecisionTreeRegressor", "GradientBoostingRegressor", "RandomForestRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}
best_features = []

for index in range (len(models)):
    model=models[index]
    
    # Ermitteln der Feature-Importance
    model.fit(X_train, y_train)
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = np.abs(model.coef_)
        
    columns = list(X_train.columns)
    feature_importance_df = pd.DataFrame({'Features':columns, 'Values':importances})
    feature_importance_df.sort_values(by='Values', ascending=False, inplace=True)
    model_spalten = feature_importance_df.loc[feature_importance_df['Values'] > 0.01].Features.tolist()
    best_features.append(model_spalten)
    
    # Trainieren des Modells mit den wichtigsten Features
    X_train_model = X_train[model_spalten]
    X_test_model = X_test[model_spalten]
    start_time = time.time()
    model.fit(X_train_model, y_train)
    end_time = time.time()
    
    y_pre=model.predict(X_test_model)

    # Berechnung der Metriken
    mse = mean_squared_error(y_test, y_pre)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pre)
    MSE.append(round(mse, 5))
    RMSE.append(round(rmse, 5))
    r_2.append(round(r2, 5))
    accuracy.append((model.score(X_test_model, y_test)) * 100)

    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')
    print("Verwendete Features: ", model_spalten)
    mse = mean_squared_error(y_test, y_pre)
    rmse = np.sqrt(mse)  # RMSE ist einfach die Quadratwurzel von MSE
    r2 = r2_score(y_test, y_pre)

    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R² Score: {r2}")
    print()
    print("--------------------------------------------------")
    print()

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time, "Best Features": best_features})

LinearRegression wurde trainiert und benötigte 0.629971981048584 Sekunden
Verwendete Features:  ['No PromoInterval', 'Promo2', 'Jan,Apr,Jul,Oct', 'Mar,Jun,Sept,Dec', 'Feb,May,Aug,Nov', 'Open', 'Month', 'Promo', 'TagnachFeiertag', 'Weekend', 'StateHoliday', 'Assortment', 'Season', 'SchoolHoliday', 'DayOfWeek', 'Quarter', 'DayOfYear', 'TagVorFeiertag', 'Year', 'Day', 'DayOfMonth', 'Promo2SinceYear', 'StoreType', 'Promo2SinceWeek', 'WeekOfYear', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Store ID', 'CompetitionDistance']
Mean Squared Error (MSE): 7168700.613016391
Root Mean Squared Error (RMSE): 2677.442924324698
R² Score: 0.5569465933428286

--------------------------------------------------

Ridge wurde trainiert und benötigte 0.038537025451660156 Sekunden
Verwendete Features:  ['Open', 'Month', 'Promo2', 'No PromoInterval', 'Promo', 'Jan,Apr,Jul,Oct', 'Mar,Jun,Sept,Dec', 'Feb,May,Aug,Nov', 'Weekend', 'StateHoliday', 'Assortment', 'Season', 'TagnachFeiertag', 'SchoolHolid

/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.824e+11, tolerance: 3.165e+08
  model = cd_fast.enet_coordinate_descent(
/Users/wladi/opt/anaconda3/envs/bv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.780e+11, tolerance: 3.165e+08
  model = cd_fast.enet_coordinate_descent(


Lasso wurde trainiert und benötigte 3.5340700149536133 Sekunden
Verwendete Features:  ['Open', 'Promo', 'Jan,Apr,Jul,Oct', 'Weekend', 'StateHoliday', 'Assortment', 'Season', 'SchoolHoliday', 'Quarter', 'Month', 'DayOfWeek', 'Feb,May,Aug,Nov', 'Year', 'Promo2SinceWeek', 'StoreType', 'WeekOfYear', 'DayOfYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'DayOfMonth', 'CompetitionOpenSinceYear', 'Store ID', 'CompetitionDistance']
Mean Squared Error (MSE): 7179624.813192917
Root Mean Squared Error (RMSE): 2679.482191243845
R² Score: 0.5562714355472267

--------------------------------------------------

DecisionTreeRegressor wurde trainiert und benötigte 0.8240931034088135 Sekunden
Verwendete Features:  ['Open', 'CompetitionDistance', 'Store ID', 'Promo', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'DayOfWeek', 'Promo2SinceYear', 'DayOfYear', 'Assortment', 'StoreType', 'No PromoInterval', 'DayOfMonth', 'Day']
Mean Squared Error (MSE): 1450082.982219882
Root Mean Squared Er

In [31]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time,Best Features
0,LinearRegression,7.168701e+06,0.55695,55.694659,2677.44292,0.629972,"[No PromoInterval, Promo2, Jan,Apr,Jul,Oct, Ma..."
1,Ridge,7.170264e+06,0.55685,55.684997,2677.73485,0.038537,"[Open, Month, Promo2, No PromoInterval, Promo,..."
2,Lasso,7.179625e+06,0.55627,55.627144,2679.48219,3.534070,"[Open, Promo, Jan,Apr,Jul,Oct, Weekend, StateH..."
3,DecisionTreeRegressor,1.450083e+06,0.91038,91.037927,1204.19391,0.824093,"[Open, CompetitionDistance, Store ID, Promo, C..."
4,GradientBoostingRegressor,4.518213e+06,0.72076,72.075695,2125.60894,11.633511,"[Open, CompetitionDistance, Promo, Store ID, P..."
5,RandomForestRegressor,8.565244e+05,0.94706,94.706348,925.48603,5.501507,"[Open, CompetitionDistance, Store ID, Promo, C..."


# RandomForestRegressor

In [40]:
X

,Store ID,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,...,Quarter,DayOfYear,DayOfMonth,Season,TagVorFeiertag,TagnachFeiertag,"Feb,May,Aug,Nov","Jan,Apr,Jul,Oct","Mar,Jun,Sept,Dec",No PromoInterval
0,1,4,1,1,0,0,1,1,570.0,11.0,...,2,120,30,2,1.0,0.0,0.0,1.0,0.0,0.0
1,1,3,1,1,0,0,1,1,570.0,11.0,...,2,119,29,2,1.0,1.0,0.0,1.0,0.0,0.0
2,1,2,1,1,0,0,1,1,570.0,11.0,...,2,118,28,2,1.0,1.0,0.0,1.0,0.0,0.0
3,1,1,1,1,0,0,1,1,570.0,11.0,...,2,117,27,2,1.0,1.0,0.0,1.0,0.0,0.0
4,1,7,0,0,0,0,1,1,570.0,11.0,...,2,116,26,2,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,1,0,0,0,1,3,9260.0,0.0,...,1,5,5,1,1.0,1.0,0.0,0.0,0.0,1.0
246899,300,5,1,0,0,1,1,3,9260.0,0.0,...,1,4,4,1,1.0,1.0,0.0,0.0,0.0,1.0
246900,300,4,1,0,0,1,1,3,9260.0,0.0,...,1,3,3,1,1.0,1.0,0.0,0.0,0.0,1.0
246901,300,3,1,0,0,1,1,3,9260.0,0.0,...,1,2,2,1,1.0,1.0,0.0,0.0,0.0,1.0


In [36]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

forest = RandomForestRegressor(random_state=42)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

columns = list(X_train.columns)
feature_importance_df = pd.DataFrame({'Features':columns, 'Values':importances})
feature_importance_df.sort_values(by='Values', ascending=False, inplace=True)
model_spalten = feature_importance_df.loc[feature_importance_df['Values'] > 0.01].Features.tolist()
print(model_spalten)

X = train_df[model_spalten]
y = train_df["Sales"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

['Open', 'CompetitionDistance', 'Store ID', 'Promo', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'DayOfWeek', 'Promo2SinceYear', 'DayOfYear', 'Assortment', 'StoreType', 'Promo2SinceWeek', 'DayOfMonth', 'Day', 'WeekOfYear']


In [ ]:
# Definieren Sie die Parameter, die Sie optimieren möchten
param_grid = {
    'n_estimators': [300],
    'max_depth': [None],
    'min_samples_split': [10]
}

# Erstellen Sie ein neues RandomForestRegressor-Objekt
forest = RandomForestRegressor(random_state=42)

# Erstellen Sie das GridSearchCV-Objekt
grid_search = GridSearchCV(forest, param_grid, cv=5, n_jobs=-1)

# Führen Sie die Suche aus
grid_search.fit(X_train, y_train)

# Drucken Sie die besten Parameter aus
print(grid_search.best_params_)


In [79]:
# # Definieren Sie die optimierten Parameter
best_params = {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 300}

# Erstellen Sie ein neues RandomForestRegressor-Objekt mit den optimierten Parametern
forest = RandomForestRegressor(max_depth=best_params['max_depth'], 
                                min_samples_split=best_params['min_samples_split'], 
                                n_estimators=best_params['n_estimators'],
                                random_state=42,
                                n_jobs=-1)
forest = RandomForestRegressor(random_state=42)
# Trainieren Sie das Modell mit den Trainingsdaten
forest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [80]:
# Machen Sie Vorhersagen auf den Testdaten
y_pred = forest.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # RMSE ist einfach die Quadratwurzel von MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 582483.1109028696
Root Mean Squared Error (RMSE): 763.2058116280756
R² Score: 0.9636429709728703


- Mean Squared Error (MSE): 997235.3682094166
- Root Mean Squared Error (RMSE): 998.616727383142
- R² Score: 0.9372307752280711

- Mean Squared Error (MSE): 861067.6367311113
- Root Mean Squared Error (RMSE): 927.9373021552218
- R² Score: 0.9460943074539725

- Mean Squared Error (MSE): 662643.2669059659
- Root Mean Squared Error (RMSE): 814.0290332082548
- R² Score: 0.9586294258103925